#### Merged Data Frame of BA (Arbeitsstellen, Arbeitslose/Arbeisuchende, Ausbildungsdaten) and Renten data

In this notebook I consolidate data that I previously scraped from the web and run preliminary analyses on them. 

I start by importing my data. BA is the data from the BA about Arbeitsstellen, Rente is the Renten data you gave me, Ausbildung is the data from the BA on Ausbildungsstellen and Bewerber and Labor_Market_Tightness is the data on Arbeitsstellen, Arbeitslose and Arbeitssuchende. The difference betweeen the BA and Labor_Market_Tightness data is that the BA data includes the Vakanzzeit and the Labor_Market_Tightness data includes not only Arbeitslose but also Arbeitssuchende. Moreover, the BA data shows the data as a floating average while the other data is the exact value for that time period. 

In [51]:
import pandas as pd

pd.set_option('display.max.rows', None)
pd.set_option('display.max.columns', None)

BA = pd.read_csv(r"C:\\Users\\jhummels\\OneDrive - DIW Berlin\\Gehlen, Annica's files - retirement-labor-shortages\\Data\\complete_df_BA.csv")
Rente = pd.read_csv(r"C:\\Users\\jhummels\\OneDrive - DIW Berlin\\Gehlen, Annica's files - retirement-labor-shortages\\Data\\complete_df_Rente.csv")
Ausbildung = pd.read_csv(r"C:\\Users\\jhummels\\OneDrive - DIW Berlin\\Gehlen, Annica's files - retirement-labor-shortages\\Data\\Ausbildungsstellen.csv")
Labor_Market_Tightness = pd.read_csv("labor_market_tightness_cleaned.csv")


Next I need to perform some data augmention, so I can merge the respective data frames into one. This includes the aggregation of monthly data to yearly data, since the retirement is only available as yearly data. For the Labor_Market_Tightness Index, I am only interested in the ratio of vacancies (Bestand an Arbeitsstellen) and job searchers (Arbeitssuchende). This ratio has been found to be a good indicator for labor market tightness and labor shortages (Abraham et. Al., 2022). To get the yearly data I take the annual mean of the monthly data. For the variables of the BA data, I also take the annual mean of the monthly data. For Ausbildungen, I just take the value in september as the yearly value, as it represents the cumulative number of Bewerber, etc. of that Berichtsjahr. Moreover, I only include years in the merged data frame for which all composite data frames have data. 

In [ ]:

##############################################################################################################
# Rename columns in the BA DataFrame
##############################################################################################################

Ausbildung.rename(columns={"Bundesland": "bland", "BKZ": "kldb2010_3_akvs", "Year": "year"}, inplace=True)
BA.rename(columns={"Bundesland": "bland", "BKZ": "kldb2010_3_akvs", "Year": "year"}, inplace=True)
Labor_Market_Tightness.rename(columns={"Bundesland": "bland", "BKZ": "kldb2010_3_akvs", "Year": "year"}, inplace=True)

##############################################################################################################
# Fix bland names
##############################################################################################################

ordered_labels =  ['1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.1', '1.11', '1.12', '1.13', '1.14',
 '1.15', '1.16', '1.17', '1.18', '1.19']
rename_map = {label: idx + 1 for idx, label in enumerate(sorted(ordered_labels))}
Labor_Market_Tightness['bland'] = Labor_Market_Tightness['bland'].astype(str).str.strip().map(rename_map)
Labor_Market_Tightness['bland'] = Labor_Market_Tightness['bland'].astype('Int64')

##############################################################################################################
# Group and aggregate to yearly data
##############################################################################################################

# Group and aggregate by taking the mean of the ratio 
valid = Labor_Market_Tightness.groupby(['year', 'bland', 'kldb2010_3_akvs']).agg(
    ratio_mean=('V/S', 'mean'),
    n_months=('V/S', 'count')
).reset_index()
valid = valid[["bland", "kldb2010_3_akvs", "year", "ratio_mean", "n_months"]]


# Group and aggregate by taking the mean of the composite columns and then taking the ratio

# List of columns to aggregate
columns_to_aggregate_tightness = [
 'Arbeitssuchende_Bestand', 'Arbeitsstellen_Bestand'
]

# Group by 'bland', 'kldb2010_3_akvs', and 'year', and aggregate the specified columns to get one value per year
# for each combination of 'bland' and 'kldb2010_3_akvs'
valid_2 = Labor_Market_Tightness.groupby(['bland', 'kldb2010_3_akvs', 'year'])[columns_to_aggregate_tightness].mean().reset_index()

valid_2['mean_V/S'] = valid_2['Arbeitsstellen_Bestand'] / valid_2['Arbeitssuchende_Bestand']

# List of columns to aggregate
columns_to_aggregate_BA = [
    'Zugang', 'Zugang_V', 'Bestand', 'Bestand_V', '3_Monate_Vakant_Anteil', '3_Monate_Vakant_V_abs',
    'abgesch_Vakanzzeit_Tage', 'abgesch_Vakanzzeit_V_abs', 'Arbeitslose',
    'Arbeitslose_V', 'Relation', 'Relation_V'
]

# Group by 'bland', 'kldb2010_3_akvs', and 'year', and aggregate the specified columns to get one value per year
# for each combination of 'bland' and 'kldb2010_3_akvs'
BA_yearly = BA.groupby(['bland', 'kldb2010_3_akvs', 'year'])[columns_to_aggregate_BA].mean().reset_index()

# Subset such that we only have data for September
Ausbildung_yearly = Ausbildung[Ausbildung['Month'] == 9]

##############################################################################################################
# Merge DataFrames on 'kldb2010_3_akvs', 'bland', and 'year'
##############################################################################################################

valid['kldb2010_3_akvs'] = valid['kldb2010_3_akvs'].astype('float64')
Ausbildung_yearly['kldb2010_3_akvs'] = Ausbildung_yearly['kldb2010_3_akvs'].astype('float64')

print("Before merge:", BA_yearly.shape[0])
merged = BA_yearly.merge(Rente, on=['kldb2010_3_akvs', 'bland', 'year'], how='left')
print("After Rente merge:", merged.shape[0])

merged = merged.merge(Ausbildung_yearly, on=['kldb2010_3_akvs', 'bland', 'year'], how='left')
print("After Ausbildung merge:", merged.shape[0])

merged = merged.merge(valid, on=['bland', 'kldb2010_3_akvs', 'year'], how='left')
print("After valid merge:", merged.shape[0])

merged = merged.merge(valid_2, on=['kldb2010_3_akvs', 'bland', 'year'], how='left')
print("After valid_2 merge:", merged.shape[0])

##############################################################################################################
# We have no Rentendata for 2011 and 2023-2025, no Ausbildungsdaten before 2014 and Rente with 63 was implemented in 2014
##############################################################################################################

filtered_merged = merged[
    (~merged['year'].isin([2011, 2012, 2013, 2014, 2023, 2024, 2025])) 
]


# Display the merged DataFrame
filtered_merged.head()

# Check for missing values in the merged DataFrame
missing_values = filtered_merged.isnull().sum()
missing_values

Then I add the tasks data to my merged data frame, so we can futher classify our Berufsgruppen by task.

In [53]:
# Load tasks data
tasks = pd.read_stata(r"C:\Users\jhummels\OneDrive - DIW Berlin\Gehlen, Annica's files - retirement-labor-shortages\Data\BA_data\C_Tasks_classification\tasks_kldb2010_3.dta")
tasks = tasks[tasks["jahr"] == 2013]

# Rename kldb2010_3 column to match your merge key
tasks.rename(columns={'kldb2010_3':'kldb2010_3_akvs'}, inplace=True)

# Merge
filtered_merged = pd.merge(
    merged,
    tasks[["kldb2010_3_akvs", "haupttask"]],  # ensure only needed columns are included
    on="kldb2010_3_akvs",
    how="left"  # or "inner" if you only want matched rows
)

Getting an overview of the data.

In [ ]:
filtered_merged.head(10)

Exporting the merged data frame. If you want the data for all years you can just export 'merged'.

In [55]:
filtered_merged.to_csv(
    r"C:\Users\jhummels\OneDrive - DIW Berlin\Gehlen, Annica's files - retirement-labor-shortages\Data\complete_data_frame.csv",
    index=False
)

Next, I standardize the data to make it comnparable across Bundesländer and Berufsgruppen. I standardize it by indexing it using 2015 as a base year: x_i_th_year / x_base_year * 100

In [ ]:
####################################################################################
# Standardize the data with base year 2015
####################################################################################
import numpy as np

# Step 1: Filter data for the base year (2015)
base_year = 2015
base_data = filtered_merged[filtered_merged['year'] == base_year]

# Step 2: Calculate the base values for each 'bland' and each numeric column
numeric_columns = filtered_merged.select_dtypes(include='number').columns.drop(['year', 'bland', 'kldb2010_3_akvs', 'counter'])  # Exclude 'year'
base_values = base_data.groupby(['bland', 'kldb2010_3_akvs'])[numeric_columns].mean().reset_index()

# Step 3: Merge the base values back into the main DataFrame
indexed_data_filtered = filtered_merged.merge(base_values, on=['bland', 'kldb2010_3_akvs'], suffixes=('', '_base'))

# Step 4: Calculate the indexed values for all numeric columns
for col in numeric_columns:
    indexed_data_filtered[f'{col}_indexed'] = (indexed_data_filtered[col] / indexed_data_filtered[f'{col}_base']) * 100

# Step 5: Drop the base value columns
indexed_data_filtered.drop(columns=[f'{col}_base' for col in numeric_columns], inplace=True)

# Step 6: Add column with logarithmic values

for col in numeric_columns:
    indexed_data_filtered[f'{col}_log'] = np.log(indexed_data_filtered[col] + 1)  # Adding 1 to avoid log(0)

# Step 5: Display the indexed data
print(indexed_data_filtered.head(5))


Next, I create lag values for each variable.

In [ ]:
# Create a new DataFrame with the lag values for each variable plot correlation heatmap 
####################################################################################

# Select numeric columns to lag (excluding identifiers)
numeric_columns = indexed_data_filtered.select_dtypes(include='number').columns.drop(['year', 'counter', 'bland', 'kldb2010_3_akvs'])

# For each numeric column, create lagged columns by -1 and -2 years
for col in numeric_columns:
    for lag in [1, 2, 3, 4]:  # 1 = lag of 1 year, 2 = lag of 2 years
        indexed_data_filtered[f'{col}_lag{lag}'] = (
            indexed_data_filtered
            .groupby(['bland', 'kldb2010_3_akvs'])[col]
            .shift(lag)
        )

Next, I export the indexed data as csv.

In [58]:
indexed_data_filtered.to_csv(
    r"C:\Users\jhummels\OneDrive - DIW Berlin\Gehlen, Annica's files - retirement-labor-shortages\Data\complete_data_frame_indexed.csv",
    index=False
)

Next, I attempt to replicate the Engpassindicator (V/S) using the BA data that uses the floating average and using the no. of unemployed rather than all job serarchers. 

In [59]:
indexed_data_filtered['V/S'] = indexed_data_filtered['Bestand'] / indexed_data_filtered['Arbeitslose']


C:\Users\jhummels\AppData\Local\Temp\ipykernel_19584\3172809519.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  indexed_data_filtered['V/S'] = indexed_data_filtered['Bestand'] / indexed_data_filtered['Arbeitslose']


I find that the two indicators are correlated with a correlation coefficient of 0.51. 

In [63]:
import numpy as np

# Drop rows where either 'V/S' or 'ratio_mean' is NaN
subset = indexed_data_filtered[['V/S', 'ratio_mean']].dropna()

# Now compute the correlation coefficient
corr_matrix = np.corrcoef(subset['V/S'], subset['ratio_mean'])
print(corr_matrix)


[[1.         0.51632026]
 [0.51632026 1.        ]]
